## Importing necessary libraries

In [1]:
import pandas as pd

## Loading Dataframe


In [2]:
dispo_alimentaire = pd.read_csv(r'D:\Elodie\Downloads\DAN-P4-FAO (1)\dispo_alimentaire.csv')
aide_alimentaire = pd.read_csv(r'D:\Elodie\Downloads\DAN-P4-FAO (1)\aide_alimentaire.csv')
population = pd.read_csv(r'D:\Elodie\Downloads\DAN-P4-FAO (1)\population.csv')
sous_nutrition = pd.read_csv(r'D:\Elodie\Downloads\DAN-P4-FAO (1)\sous_nutrition.csv')

## PreProcessing

In [3]:
population['Valeur'] = population['Valeur'] * 1000
dispo_alimentaire.fillna(0)
sous_nutrition['Valeur'] = pd.to_numeric(sous_nutrition['Valeur'], errors='coerce')
sous_nutrition.fillna(0)
sous_nutrition['Valeur'] = sous_nutrition['Valeur'] * 1000000

## Questions de Marc

### 1) la proportion de personnes en état de sous-nutrition

In [4]:
pop_sous_nutrition = pd.merge(population.loc[population['Année'] == 2017], sous_nutrition.loc[sous_nutrition['Année'] == '2016-2018'], on='Zone', suffixes=('_population_totale', '_population_sous_nutrie'))
taux_sous_nutrition = (pop_sous_nutrition['Valeur_population_sous_nutrie'].sum() / pop_sous_nutrition['Valeur_population_totale'].sum() * 100).round(2)
f'la proportion de personnes en état de sous nutrition est de {taux_sous_nutrition} %'

'la proportion de personnes en état de sous nutrition est de 7.1 %'

### 2) le nombre théorique de personnes qui pourraient être nourries. Tu devrais pouvoir calculer ça à partir de la disponibilité alimentaire mondiale 

In [5]:
# selon le site de la FAO, un humain a besoin de 2720 Kcal jounralière
besoin_alim_monde17 = int(2720 * 365 * pop_sous_nutrition['Valeur_population_totale'].sum())
dispo_alimentaire_kcal = pd.merge(population.loc[population['Année'] == 2017], dispo_alimentaire.groupby(["Zone"]).sum(), on='Zone')
dispo_alim_monde17 = dispo_alimentaire_kcal['Disponibilité alimentaire (Kcal/personne/jour)'] * dispo_alimentaire_kcal['Valeur'] * 365
f'le nombre theorique de personnes pouvant etre nourries est de {int(dispo_alim_monde17.sum()/(2720*365))} personnes soit {int(dispo_alim_monde17.sum()/besoin_alim_monde17*100)} % des besoins alimentaires de la planète.'

'le nombre theorique de personnes pouvant etre nourries est de 7690803171 personnes soit 101 % des besoins alimentaires de la planète.'

### 3) idem pour la disponibilité alimentaire des produits végétaux

In [6]:
veggie = dispo_alimentaire.loc[dispo_alimentaire['Origine'] == "vegetale"]
dispo_veg17 = pd.merge(population.loc[population['Année'] == 2017], veggie.groupby(["Zone"]).sum(), on='Zone')
dispo_veg_kcal = (dispo_veg17['Disponibilité alimentaire (Kcal/personne/jour)']) * dispo_veg17['Valeur'] * 365
f'le nombre theorique de personnes pouvant etre nourries est de {int(dispo_veg_kcal.sum()/(2720*365))} personnes soit {int(dispo_veg_kcal.sum()/besoin_alim_monde17*100)} % des besoins alimentaires de la planète.'

'le nombre theorique de personnes pouvant etre nourries est de 6345869195 personnes soit 84 % des besoins alimentaires de la planète.'

### 4) l’utilisation de la disponibilité intérieure, en particulier la part qui est attribuée à l’alimentation animale, celle qui est perdue et celle qui est concrètement utilisée pour l'alimentation humaine

In [7]:
#Disponibilité intérieure = Semences + Pertes + Nourriture + Aliments pour animaux + Traitement + Autres utilisations
#Disponibilité intérieure = Production + Importations - Exportations + Variation de stock

dispo_interieur = dispo_alimentaire["Disponibilité intérieure"].sum() 
part_animal = (dispo_alimentaire[('Aliments pour animaux')].sum()   / dispo_interieur * 100).round(2)
part_pertes = (dispo_alimentaire[('Pertes')].sum()   / dispo_interieur * 100).round(2)
part_humaine = (dispo_alimentaire[('Nourriture')].sum()   / dispo_interieur * 100).round(2)
f'la part de la disponibilité intérieure utilisé pour l alimentation animale est {part_animal} %, la part perdue est de {part_pertes} % et la part utilisée pour l alimentation humaine est {part_humaine} %'

'la part de la disponibilité intérieure utilisé pour l alimentation animale est 13.24 %, la part perdue est de 4.61 % et la part utilisée pour l alimentation humaine est 49.51 %'

### Lors de mes recherches, j’ai découvert des chiffres intéressants sur l’utilisation des céréales, notamment la répartition entre l’alimentation humaine (colonne Nourriture) et l’alimentation pour animaux. 

In [8]:
cereales = ["Blé et produits", "Riz et produits", "Orge et produits", "Maïs et produits", "Seigle et produits", "Avoine", "Millet et produits", "Sorgho et produits", "Céréales, Autres"]
dispo_cereale = dispo_alimentaire.loc[dispo_alimentaire['Produit'].isin(cereales)] 
dispo_cereale_interieur = dispo_cereale["Disponibilité intérieure"].sum() 
part_cereale_animal = (dispo_cereale[('Aliments pour animaux')].sum()   / dispo_cereale_interieur * 100).round(2)
part_cereale_humaine = (dispo_cereale[('Nourriture')].sum()   / dispo_cereale_interieur * 100).round(2)
f'La proportion de l alimentation humaine est de {part_cereale_humaine} % et la proportion de l alimentation animale est de {part_cereale_animal} %'

'La proportion de l alimentation humaine est de 18.13 % et la proportion de l alimentation animale est de 69.34 %'

### je me souviens d’un exemple étonnant d’utilisation du manioc par la Thaïlande aux égards de la proportion de personnes en sous-nutrition. Il serait intéressant d’aller à nouveau y jeter un coup d’œil, de mémoire ça concernait l’exportation par rapport à la production.

In [9]:
thai = pop_sous_nutrition.loc[pop_sous_nutrition['Zone'] == 'Thaïlande']
thai_propor = int(thai['Valeur_population_sous_nutrie'] / thai['Valeur_population_totale'] * 100)
f'La proportion de la population Thailandaise en état de sous nutrition est de {thai_propor} %.'

'La proportion de la population Thailandaise en état de sous nutrition est de 8 %.'

In [10]:
thai_alim = dispo_alimentaire.loc[dispo_alimentaire['Zone'] == 'Thaïlande'] 
manioc_thailande = thai_alim.loc[thai_alim['Produit'] == 'Manioc'] 
manioc_thailande[['Zone','Exportations - Quantité','Production']]

,Zone,Exportations - Quantité,Production
13809,Thaïlande,25214.0,30228.0


In [11]:
prod_export_thai = int(manioc_thailande['Exportations - Quantité'] / manioc_thailande['Production'] * 100)
f' {prod_export_thai} % de la production thailandaise de manioc est exportée.'

' 83 % de la production thailandaise de manioc est exportée.'

## Questions de Mélanie

### 1) j’aimerais que tu puisses nous donner les pays pour lesquels la proportion de personnes sous-alimentées est la plus forte en 2017

In [12]:
pop_sous_nutrition['prop_sous_nutri'] = pop_sous_nutrition['Valeur_population_sous_nutrie'] / pop_sous_nutrition['Valeur_population_totale'] * 100
(pop_sous_nutrition[['Zone', 'prop_sous_nutri']].sort_values(['prop_sous_nutri'], ascending=[False])).head(10).round(2)

,Zone,prop_sous_nutri
78,Haïti,48.26
157,République populaire démocratique de Corée,47.19
108,Madagascar,41.06
103,Libéria,38.28
100,Lesotho,38.25
183,Tchad,37.96
161,Rwanda,35.06
121,Mozambique,32.81
186,Timor-Leste,32.17
0,Afghanistan,28.93


### 2) ceux qui ont le plus bénéficié d’aide depuis 2013

In [13]:
benef_aide = aide_alimentaire.groupby(["Pays bénéficiaire"]).sum()
benef_aide[['Valeur']].sort_values(['Valeur'], ascending=[False]).head(10)

,Valeur
Pays bénéficiaire,
République arabe syrienne,1858943
Éthiopie,1381294
Yémen,1206484
Soudan du Sud,695248
Soudan,669784
Kenya,552836
Bangladesh,348188
Somalie,292678
République démocratique du Congo,288502


### 3) ceux ayant le plus/le moins de disponibilité/habitant

In [14]:
dispo_habitant = dispo_alimentaire.groupby(["Zone"]).sum()
dispo_habitant[['Disponibilité alimentaire (Kcal/personne/jour)']].sort_values(['Disponibilité alimentaire (Kcal/personne/jour)'], ascending=[False]).head(10)

,Disponibilité alimentaire (Kcal/personne/jour)
Zone,
Autriche,3770.0
Belgique,3737.0
Turquie,3708.0
États-Unis d'Amérique,3682.0
Israël,3610.0
Irlande,3602.0
Italie,3578.0
Luxembourg,3540.0
Égypte,3518.0


In [15]:
dispo_habitant[['Disponibilité alimentaire (Kcal/personne/jour)']].sort_values(['Disponibilité alimentaire (Kcal/personne/jour)']).head(10)

,Disponibilité alimentaire (Kcal/personne/jour)
Zone,
République centrafricaine,1879.0
Zambie,1924.0
Madagascar,2056.0
Afghanistan,2087.0
Haïti,2089.0
République populaire démocratique de Corée,2093.0
Tchad,2109.0
Zimbabwe,2113.0
Ouganda,2126.0
